### Dividend Based Investing
A model that ranks stocks based on their dividend strength using weighted normalized metrics.

Import the libraries

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import math

Load The Stocks List

In [2]:
tickers = pd.read_csv('top50_us_stocks_info.csv')
tickers.head()

,Name,Ticker,Sector,MarketCap,PreviousClose,52WeekHigh,52WeekLow
0,3M Company,MMM,Industrials,87442300928,160.59,172.85,121.98
1,A.O. Smith Corporation,AOS,Industrials,9207796736,65.15,77.92,58.83
2,Abbott Laboratories,ABT,Healthcare,217098158080,125.80,141.23,110.86
3,AbbVie Inc.,ABBV,Healthcare,383705284608,215.89,244.81,163.81
4,Accenture plc,ACN,Technology,153862651904,242.90,398.35,229.40


Create Dividend DataFrame

In [3]:
def create_dividend_df(tickers): 
    # Define the columns to store dividend-related data for each stock
    cols = [
        'Ticker',
        'Dividend Yield(%)',
        'Dividend Rate',
        'Payout Ratio',
        'Five Year Avg Dividend Yield(%)',
        'Earning Growth(%)'
    ]

    # Create an empty df with the above columns
    dividend_df = pd.DataFrame(columns=cols)

    for stock in tickers:
        ticker = yf.Ticker(stock)
        info = ticker.info

        # Extract key dividend metrics (convert ratios to % where needed)
        dividend_yield = info.get('dividendYield', np.nan) * 100 if info.get('dividendYield') else np.nan
        dividend_rate = info.get('dividendRate', np.nan)
        payout_ratio = info.get('payoutRatio', np.nan) * 100 if info.get('payoutRatio') else np.nan
        five_year_avg_dividend_yield = info.get('fiveYearAvgDividendYield', np.nan) if info.get('fiveYearAvgDividendYield') else np.nan
        earnings_growth = info.get('earningsGrowth', np.nan) * 100 if info.get('earningsGrowth') else np.nan

        # Append the extracted data as a new row in the df
        dividend_df.loc[len(dividend_df)] = [
            stock,
            dividend_yield,
            dividend_rate,
            payout_ratio,
            five_year_avg_dividend_yield,
            earnings_growth
        ]

    # Columns that will be normalized later
    numeric_cols = {
        'Dividend Yield(%)',
        'Dividend Rate',
        'Payout Ratio',
        'Five Year Avg Dividend Yield(%)',
        'Earning Growth(%)'
    }

    # Define weightage for each normalized metric (used for computing final score)
    # These can be adjusted based on data analysis or investor preference
    weights = {
        'Dividend Yield(%) Normalised': 0.3,
        'Dividend Rate Normalised': 0.2,
        'Payout Ratio Normalised': 0.2,
        'Five Year Avg Dividend Yield(%) Normalised': 0.2,
        'Earning Growth(%) Normalised': 0.1
    }

    # Normalize numeric columns using Min-Max normalization:
    # Normalized value = (value - min) / (max - min)
    for col in numeric_cols :
        dividend_df[col + ' Normalised'] = (dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min())
        
        # For payout ratio, higher values are worse, so we invert the normalized score
        if col == 'Payout Ratio':
            dividend_df[col + ' Normalised'] = 1 - dividend_df[col + ' Normalised']

    return dividend_df

In [4]:
# Convert the 'Ticker' col from the CSV into a list of stock tickers
tickers_list = tickers['Ticker'].values.tolist()
dividend_df = create_dividend_df(tickers_list)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio,Five Year Avg Dividend Yield(%),Earning Growth(%),Dividend Rate Normalised,Dividend Yield(%) Normalised,Payout Ratio Normalised,Earning Growth(%) Normalised,Five Year Avg Dividend Yield(%) Normalised
0,MMM,178.0,2.92,46.090000,4.03,-37.5,0.267481,0.158511,0.944231,0.075807,0.510444
1,AOS,209.0,1.38,36.660000,1.72,14.6,0.096559,0.191489,0.958070,0.152946,0.208877
2,ABT,189.0,2.36,29.150000,1.71,NaN,0.205327,0.170213,0.969092,NaN,0.207572
3,ABBV,319.0,6.92,490.149970,3.78,-88.7,0.711432,0.308511,0.292524,0.000000,0.477807
4,ACN,263.0,6.52,48.720002,1.45,-15.5,0.667037,0.248936,0.940371,0.108380,0.173629
5,ADBE,NaN,NaN,NaN,0.12,11.2,NaN,NaN,NaN,0.147912,0.000000
6,AMD,NaN,NaN,NaN,NaN,60.3,NaN,NaN,NaN,0.220610,NaN
7,AES,495.0,0.70,46.080002,3.55,25.3,0.021088,0.495745,0.944245,0.168789,0.447781
8,AFL,209.0,2.32,29.430000,2.20,-64.2,0.200888,0.191489,0.968681,0.036275,0.271540
9,A,68.0,0.99,22.900000,0.66,21.6,0.053274,0.041489,0.978265,0.163311,0.070496


Compute Dividend Score

In [5]:
# Define weightage for each normalized dividend-related metric
# These weights determine the importance of each factor in the overall score
weights = {
    'Dividend Yield(%) Normalised': 0.3,
    'Dividend Rate Normalised': 0.2,
    'Payout Ratio Normalised': 0.2,
    'Five Year Avg Dividend Yield(%) Normalised': 0.2,
    'Earning Growth(%) Normalised': 0.1
}

# Calculate the final 'Dividend Score' for each stock:
# → multiply each normalized col by its corresponding weight
# → sum across all weighted cols to get a single composite score per stock
dividend_df['Dividend Score'] = dividend_df[[col for col in weights.keys()]].mul(list(weights.values())).sum(axis = 1)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio,Five Year Avg Dividend Yield(%),Earning Growth(%),Dividend Rate Normalised,Dividend Yield(%) Normalised,Payout Ratio Normalised,Earning Growth(%) Normalised,Five Year Avg Dividend Yield(%) Normalised,Dividend Score
0,MMM,178.0,2.92,46.090000,4.03,-37.5,0.267481,0.158511,0.944231,0.075807,0.510444,0.399565
1,AOS,209.0,1.38,36.660000,1.72,14.6,0.096559,0.191489,0.958070,0.152946,0.208877,0.325443
2,ABT,189.0,2.36,29.150000,1.71,NaN,0.205327,0.170213,0.969092,NaN,0.207572,0.327462
3,ABBV,319.0,6.92,490.149970,3.78,-88.7,0.711432,0.308511,0.292524,0.000000,0.477807,0.388906
4,ACN,263.0,6.52,48.720002,1.45,-15.5,0.667037,0.248936,0.940371,0.108380,0.173629,0.441726
5,ADBE,NaN,NaN,NaN,0.12,11.2,NaN,NaN,NaN,0.147912,0.000000,0.014791
6,AMD,NaN,NaN,NaN,NaN,60.3,NaN,NaN,NaN,0.220610,NaN,0.022061
7,AES,495.0,0.70,46.080002,3.55,25.3,0.021088,0.495745,0.944245,0.168789,0.447781,0.448225
8,AFL,209.0,2.32,29.430000,2.20,-64.2,0.200888,0.191489,0.968681,0.036275,0.271540,0.349296
9,A,68.0,0.99,22.900000,0.66,21.6,0.053274,0.041489,0.978265,0.163311,0.070496,0.249185


Rank Stocks

In [6]:
# Sort all stocks in descending order based on their 'Dividend Score'
dividend_df = dividend_df.sort_values(by='Dividend Score', ascending=False)

# Reset the index after sorting to maintain a clean, continuous order
dividend_df.reset_index(drop=True, inplace=True)
dividend_df.head(10)

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio,Five Year Avg Dividend Yield(%),Earning Growth(%),Dividend Rate Normalised,Dividend Yield(%) Normalised,Payout Ratio Normalised,Earning Growth(%) Normalised,Five Year Avg Dividend Yield(%) Normalised,Dividend Score
0,MO,742.0,4.24,78.629994,7.78,5.2,0.413984,0.758511,0.896475,0.139029,1.000000,0.703548
1,AMGN,298.0,9.52,72.620004,3.07,13.6,1.000000,0.286170,0.905295,0.151466,0.385117,0.559080
2,T,452.0,1.11,36.160000,6.63,NaN,0.066593,0.450000,0.958804,NaN,0.849869,0.510053
3,ARE,969.0,5.28,689.469960,3.91,NaN,0.529412,1.000000,0.000000,NaN,0.494778,0.504838
4,ALL,205.0,4.00,12.710000,2.35,586.7,0.387347,0.187234,0.993220,1.000000,0.291123,0.490508
5,AMT,379.0,6.80,107.180000,2.71,NaN,0.698113,0.372340,0.854575,NaN,0.338120,0.489864
6,APD,301.0,7.16,100.990000,2.35,2.4,0.738069,0.289362,0.863659,0.134883,0.291123,0.478867
7,ADM,362.0,2.04,87.830000,2.68,489.1,0.169811,0.354255,0.882973,0.855493,0.334204,0.469223
8,AMCR,632.0,0.51,158.590000,4.62,NaN,0.000000,0.641489,0.779125,NaN,0.587467,0.465765
9,AEP,317.0,3.80,54.470000,3.57,0.6,0.365150,0.306383,0.931932,0.132218,0.450392,0.454631


Determine Position Size

In [ ]:
# Get total investment amount from user
portfolio_size = int(input("Enter the investment amount: "))

# Calculate equal position size per stock
position_size = portfolio_size / len(dividend_df.index)
position_size

Calculate Number of Shares

In [ ]:
# Fetch latest price for each ticker
dividend_df['Latest Price'] = dividend_df['Ticker'].apply(
    lambda x: yf.Ticker(x).history(period='1d')['Close'].iloc[-1]
)

# Calc number of shares to buy for each stock based on position size
# Use floor to avoid fractional shares
dividend_df['Number of Shares to buy'] = dividend_df['Latest Price'].apply(lambda price: math.floor( 
    position_size / price
))
dividend_df